In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Environnement

In [ ]:
#@title Environnement
class Env:

    def __init__(self,size):
        self.size=size
        self.maze=np.ones([size,size],dtype=np.int)

        self.maze[3,1]=10
        self.maze[3,3]=10
        self.maze[5,7]=10
        self.maze[7,7]=10

        self.passage_count=np.zeros([size,size])
        self.state=0

        self.reset()

    def state_to_pos(self,state):
        if state is None:
            return None
        return state//self.size , state%self.size

    def pos_to_state(self,pos):
        if pos is None:
            return None
        return pos[0]*self.size+pos[1]


    def reset(self):
        self.state=0
        self.passage_count[0,0]+=1
        return self.state

    def render(self):
        fig, ax = plt.subplots()
        ax.matshow(self.passage_count,cmap=plt.cm.Blues)

        for i in range(self.size):
            for j in range(self.size):
                ax.text(j, i, str(self.maze[i,j]), va='center', ha='center')

    def step(self,action):

        assert self.state is not None, "on ne peut pas bouger quand l'état est None"

        pos=self.state_to_pos(self.state)

        if action==0:
            new_pos=(pos[0]+1,pos[1])
        else:
            new_pos=(pos[0],pos[1]+1)

        done = new_pos[0]>=self.size or new_pos[1]>=self.size

        if done:
            if self.state==self.size*self.size-1: #arrivé à destination
                reward=20
            else: #tombé sur le côté
                reward=-5
            self.state=None
        else:
            self.passage_count[new_pos]+=1
            reward=self.maze[new_pos]
            self.state=self.pos_to_state(new_pos)

        return self.state, reward , done


env=Env(10)

for _ in range(3):
    done=False
    while not done:
        action=np.random.randint(2)
        _,_,done=env.step(action)
    env.reset()

env.render()

Voici un environnement constitué de size $\times$ size=10 $\times$ 10 états. Chaque état est associé à une positions `(i,j)`. Par exemple:

* L'état 0 est associé à la position (0,0)
* L'état 1 est associé à la position (0,1)

* L'état 10 est associé à la position (1,0)

* L'état 11 est associé à la position (1,1)


In [ ]:
env.state_to_pos(0)

In [ ]:
env.state_to_pos(1)

In [ ]:
env.state_to_pos(10)

In [ ]:
env.state_to_pos(11)


La valeur de `maze[i,j]` donne la réconpense que l'on reçoit si l'on va en cette position `i,j`. Certaines positions sont plus avantageuses:

In [ ]:
env.maze

Depuis chaque état on peut effetuer 2 actions:

* action 0 = descendre
* action 1 = aller à droite

In [ ]:
env.reset()
print("avant action, state=",env.state)
next_state,rewards,done=env.step(1)
print("après action, state=",env.state)

In [ ]:
env.reset()
print("avant action, state=",env.state)
next_state,rewards,done=env.step(0)
print("après action, state=",env.state)



Notons que lorsque une `action` fait sortir la position du carré [0,...,9] × [0,...,9], le jeu s'arête. La variable `done` renvoyé par `env.step(action)`  est alors `True`.

In [ ]:
env.state=9 #=> position (0,9), dans le coin en haut à droite
next_state,rewards,done=env.step(1)
next_state,rewards,done

***A vous:*** Il y a aussi des récompenses qui ne sont pas stockées dans la matrice `maze`, lesquelles ?

Pour finir,
* La méthode `env.reset()` permet de repartir de l'état 0 (le point de départ)

* la méthode `env.render()` permet de visualiser l'ensemble des parcours effectués depuis la création de l'objet `env` (donc plusieur parcours si on a lancé plusieur fois la méthode `reset`).

### L'algo











Le but du projet est d'implémenter l'algo `SARSA` qui permette de trouver le/les parcours optimaux pour ramasser le plus de récompenses.


* On part d'un état $s$ initial
* On choisit une action $a\in \{0,1\}$ suivant  l'$\epsilon$-stratégie, ce qui conduit à  l'état $s'$ et donne la récompense $r$.
* On modifie la matrice $Q$:
$$
Q[s,a] \leftarrow  r + \gamma \max_{a'} Q[s',a']
$$
et si $s'$ est un état terminal:
$$
Q[s,a] \leftarrow  r
$$
* si $s$ n'est pas terminal, on repart de la première étape avec $s\leftarrow s'$.
* sinon, l'épisode est fini, on diminue $\epsilon$ et on repart du $s$  initial (toujours égal à 0 dans ce jeu).




Vous pourrez aussi implémenter la variante avec inertie:
$$
Q[s,a] \leftarrow (1-\alpha)Q[s,a]  + \alpha(  r + \gamma \max_{a'} Q[s',a'] )
$$


Exemple de paramètres qui fonctionnent.

    epsilon = 0.9
    alpha = 0.5
    gamma = 0.95

Chez moi,  5_000 épisodes suffisent à faire converger l'algo.

Faites décroitre ϵ très lentement.